In [1]:
import os
os.environ['NUMBA_ENABLE_CUDASIM'] = "1"

# playing with porting to CUDA

In [2]:
import numba.cuda as cuda
import numba as nb
import numpy as np
import math
from math import asinh, sqrt, atan
from time import time

In [3]:
eps = 1e-10

@cuda.jit
def increment_a_3D_array(an_array):
 rray.shape[2] + z

n = (20, 10, 1)
an_array = np.zeros(( 2*n[0]-1, 2*n[1] - 1, 2 * n[2]-1)) #, 2*n[2] - 1))

threadsperblock = (8, 8, 8)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid_z = math.ceil(an_array.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)
increment_a_3D_array[blockspergrid, threadsperblock](an_array)   x, y, z = cuda.grid(3)
    if x < an_array.shape[0] and y < an_array.shape[1] and z < an_array.shape[2]:
       an_array[x, y, z] = x * (an_array.shape[1]* an_array.shape[2]) + y * an_a

In [ ]:
print(an_array[-1 ,-1, 0])

# Optimizing Newell f and g functions

## f and g orignal

In [4]:
# newell f
def f_orig(p):
  #print(type(p))
  x, y, z = abs(p[0]), abs(p[1]), abs(p[2])
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
def g_orig(p):
  x, y, z = p[0], p[1], abs(p[2])
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

%timeit f_orig((1e-9, 1e-9, 1e-9))
%timeit g_orig((1e-9, 1e-9, 1e-9))

NameError: name 'eps' is not defined

## f and g in jit

In [ ]:
# newell f
from numba import jit, float64

@jit(float64(float64, float64, float64),nopython=True)
def f_jit(x, y, z):
  #print(type(p))
  x, y, z = abs(x), abs(y), abs(z)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@jit(float64(float64, float64, float64), nopython=True)
def g_jit(x,y,z):
  z = abs(z)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

%timeit f_jit(1e-9, 1e-9, 1e-9)
%timeit g_jit(1e-9, 1e-9, 1e-9)

In [ ]:
# newell f
from numba import jit, float64
import numpy as np

@jit(float64(float64, float64, float64), nopython=True)
def f_jit(x, y, z):
  #print(type(p))
  x, y, z = np.abs(x), np.abs(y), np.abs(z)
  return + y / 2.0 * (z**2 - x**2) * np.arcsinh(y / (np.sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * np.arcsinh(z / (np.sqrt(x**2 + y**2) + eps)) \
         - x*y*z * np.arctan(y*z / (x * np.sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * np.sqrt(x**2 + y**2 + z**2)

# newell g
@jit(float64(float64, float64, float64), nopython=True)
def g_jit(x,y,z):
  z = np.abs(z)
  return + x*y*z * np.arcsinh(z / (np.sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * np.arcsinh(x / (np.sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * np.arcsinh(y / (np.sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * np.arctan(x*y / (z * np.sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * np.arctan(x*z / (y * np.sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * np.arctan(y*z / (x * np.sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * np.sqrt(x**2 + y**2 + z**2) / 3.0

%timeit f_jit(1e-9, 1e-9, 1e-9)
%timeit g_jit(1e-9, 1e-9, 1e-9)

# f and g in cfunc

In [ ]:
from numba import cfunc

@cfunc(float64(float64, float64, float64), nopython=True)
def f_cfunc(x, y, z):
  #print(type(p))
  x, y, z = np.abs(x), np.abs(y), np.abs(z)
  return + y / 2.0 * (z**2 - x**2) * np.arcsinh(y / (np.sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * np.arcsinh(z / (np.sqrt(x**2 + y**2) + eps)) \
         - x*y*z * np.arctan(y*z / (x * np.sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * np.sqrt(x**2 + y**2 + z**2)

# newell g
@cfunc(float64(float64, float64, float64), nopython=True)
def g_cfunc(x,y,z):
  z = np.abs(z)
  return + x*y*z * np.arcsinh(z / (np.sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * np.arcsinh(x / (np.sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * np.arcsinh(y / (np.sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * np.arctan(x*y / (z * np.sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * np.arctan(x*z / (y * np.sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * np.arctan(y*z / (x * np.sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * np.sqrt(x**2 + y**2 + z**2) / 3.0

# very sad result
%timeit f_cfunc(1e-9, 1e-9, 1e-9)
%timeit g_cfunc(1e-9, 1e-9, 1e-9)

# GPU with numba jit

In [ ]:
# newell f
@cuda.jit
def f_cuda(x,y,z):
  #print(type(p))
  x, y, z = abs(x), abs(y), abs(z)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@cuda.jit
def g_cuda(x,y,z):
  z =  abs(z)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

@cuda.jit
def array_3D_set_f_g(an_array):
    x, y, z = cuda.grid(3)
    if x < an_array.shape[0] and y < an_array.shape[1] and z < an_array.shape[2]:
       an_array[x, y, z] = f_cuda(x,y,z)

n = (125, 25, 1)
an_array = np.zeros(( 2*n[0]-1, 2*n[1] - 1, 2 * n[2]-1))
arr = cuda.to_device(an_array)

threadsperblock = (16, 16, 4)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid_z = math.ceil(an_array.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)

%timeit array_3D_set_f_g[blockspergrid, threadsperblock](arr)

## same as above (but with trig functions from numpy)

In [ ]:
# newell f
@cuda.jit(fastmath=False)
def f_cuda(x,y,z):
  x, y, z = abs(x), abs(y), abs(z)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@cuda.jit(fastmath=False)
def g_cuda(x,y,z):
  z =  abs(z)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

@cuda.jit
def array_3D_set_f_g(an_array, idxes):
    x, y, z = cuda.grid(3)
    #print(x,y,z)
    if x < an_array.shape[0] and y < an_array.shape[1] and z < an_array.shape[2]:
       an_array[x, y, z] = f_cuda(x,y,z)

n = (125, 25, 1)
an_array = np.zeros(( 2*n[0]-1, 2*n[1] - 1, 2 * n[2]-1), dtype=np.float64)
arr = cuda.to_device(an_array)
idxes = np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6)
idxes = cuda.to_device(idxes)

threadsperblock = (16, 16, 4)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid_z = math.ceil(an_array.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)

array_3D_set_f_g[blockspergrid, threadsperblock](arr, idxes)

# Tests on demag calculation

In [ ]:
from math import pi
from time import time

# demag tensor setup
def set_n_demag_orig(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      vec = list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute))
      value += (-1)**sum(i) * func((vec[0], vec[1], vec[2]))
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()

def identity(p):
    return 0

def calculate_demag_tensor_orig(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])

    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag_orig(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    return n_demag

n = (125, 25, 1)
dx = (1e-9, 1e-9, 1e-9)
t = time()

f = f_orig
g = g_orig

tensor_original = calculate_demag_tensor_orig(n ,dx)
print(f"Took {time() - t} s")

In [ ]:
from math import pi
from time import time

@jit(float64(float64, float64, float64),nopython=True)
def f_jit(x, y, z):
  #print(type(p))
  x, y, z = abs(x), abs(y), abs(z)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@jit(float64(float64, float64, float64), nopython=True)
def g_jit(x,y,z):
  z = abs(z)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

# demag tensor setup
def set_n_demag(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      vec = list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute))
      value += (-1)**sum(i) * func(vec[0], vec[1], vec[2])
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()

def identity(p):
    return 0

def calculate_demag_tensor(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])

    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    return n_demag

n = (125, 25, 1)
dx = (1e-9, 1e-9, 1e-9)
t = time()

f = f_jit
g = g_jit

tensor_f_g_jit = calculate_demag_tensor(n ,dx)
print(f"Took {time() - t} s")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print("differences between the values in tensors")
to = tensor_original[:,:,0,:]
tjit = tensor_f_g_jit[:,:,0,:]
diff = np.abs((to - tjit))

#diff[diff != 0] /= to[diff != 0]

#for x in diff.shape[0]:
#    for y in diff.shape[1]:
#        for z in diff.shape[2]:
#            if diff[x,y,z] != 0:
#                diff[x,y,z] =
print(f"mean absolute difference : {np.sum(diff) / np.sum(to != 0)}")

for i in range(6):
    sns.heatmap(diff[:, :, i])
    plt.show()

In [3]:
# demag tensor setup
def set_n_demag_verbose(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = [(it.multi_index[0] + n[0] - 1) % (2*n[0] - 1) - n[0] + 1,
             (it.multi_index[1] + n[1] - 1) % (2*n[1] - 1) - n[1] + 1,
             (it.multi_index[2] + n[2] - 1) % (2*n[2] - 1) - n[2] + 1]
      #print(it.multi_index[0], it.multi_index[1], it.multi_index[2])
      x = (idx[permute[0]] + i[permute[0]] - i[permute[0]+3]) * dx[permute[0]]
      y = (idx[permute[1]] + i[permute[1]] - i[permute[1]+3]) * dx[permute[1]]
      z = (idx[permute[2]] + i[permute[2]] - i[permute[2]+3]) * dx[permute[2]]
      value += (-1)**sum(i) * func((x,y,z))
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()

def identity(p):
    return 0

def calculate_demag_tensor_verbose(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])

    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag_verbose(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    return n_demag


n = (2, 1, 1)
dx = (1e-9, 1e-9, 1e-9)
t = time()
f = f_orig
g = g_orig

verbose = calculate_demag_tensor_verbose(n ,dx)
print(f"Took {time() - t} s")
print(verbose)

NameError: name 'f_orig' is not defined

In [ ]:
print("differences between the values in tensors")
to = tensor_original[:,:,0,:]
tvg = verbose[:,:,0,:]
diff = np.abs((to - tvg))
print(f"mean absolute difference : {np.sum(diff)}")

# Demag on GPU

In [104]:
# newell f
#@cuda.jit

eps=1e-10

@cuda.jit
def f_cuda(x,y,z):
  #print(type(p))
  x, y, z = abs(x), abs(y), abs(z)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@cuda.jit
def g_cuda(x,y,z):
  z =  abs(z)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0

@cuda.jit
def demag_calc_gpu(array, idxes, n, permute, dx, _func, idx_table):
    x_idx, y_idx, z_idx = cuda.grid(3)
    if x_idx < array.shape[0] and y_idx < array.shape[1] and z_idx < array.shape[2]:
        idx = idx_table[x_idx, y_idx, z_idx,:]
        value = 0
        i = 0
        while i < 64:
            idx[ 0] = (x_idx + n[0] - 1) % (2*n[0] - 1) - n[0] + 1
            idx[ 1] = (y_idx + n[1] - 1) % (2*n[1] - 1) - n[1] + 1
            idx[ 2] = (z_idx + n[2] - 1) % (2*n[2] - 1) - n[2] + 1


            x = (idx[permute[0]] + idxes[i][permute[0]] - idxes[i][permute[0]+3]) * dx[permute[0]]
            y = (idx[permute[1]] + idxes[i][permute[1]] - idxes[i][permute[1]+3]) * dx[permute[1]]
            z = (idx[permute[2]] + idxes[i][permute[2]] - idxes[i][permute[2]+3]) * dx[permute[2]]

            sign = -1**(idxes[i][0] + idxes[i][1] + idxes[i][2] + idxes[i][3] + idxes[i][4] + idxes[i][5])

            if _func[0] == 0:
                value += sign * f_cuda(x,y,z)
            else:
                value += sign * g_cuda(x,y,z)

            i += 1
        div = 12.56637061 * dx[0] * dx[1] * dx[2]
        array[x_idx,y_idx,z_idx] = - value / div

# demag tensor setup
def set_n_demag_cuda(permute, func, dx):
    threadsperblock = (32, 8, 1)
    an_array = np.zeros(( 2*n[0]-1, 2*n[1] - 1, 2 * n[2]-1), dtype=np.float64)
    arr_cuda = cuda.to_device(an_array)
    n_cuda = cuda.to_device(np.array(n, dtype=int))
    permute_cuda = cuda.to_device(np.array(permute, dtype=int))
    dx_cuda = cuda.to_device(np.array(dx, dtype=np.float64))
    idxes = np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6)
    idxes_cuda = cuda.to_device(idxes)
    idx_cuda = np.zeros(an_array.shape + (3, ))
    idx_cuda = cuda.to_device(idx_cuda)

    if func == 'f':
        #print(0)
        _func = np.array([0])
    else:
        #print(1)
        _func = np.array([1])

    blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
    blockspergrid_z = math.ceil(an_array.shape[2] / threadsperblock[2])
    blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)

    demag_calc_gpu[blockspergrid, threadsperblock](arr_cuda, idxes_cuda, n_cuda, permute_cuda, dx_cuda, _func, idx_cuda)
    return arr_cuda

def identity(p):
    return 0

def calculate_demag_tensor_cuda(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])
    res = []

    for i, t in enumerate((('f',0,1,2),('g',0,1,2),('g',0,2,1),('f',1,2,0),('g',1,2,0),('f',2,0,1))):
        r = set_n_demag_cuda(func=t[0], permute=t[1:], dx=dx).copy_to_host()
        res.append(np.expand_dims(r, 3))
    return np.concatenate(res, axis=3)

n = (2, 1, 1)
dx = (1e-9, 1e-9, 1e-9)
t = time()
f = f_cuda
g = g_cuda

cuda_calc = calculate_demag_tensor_cuda(n ,dx)
print(cuda_calc.shape)
print(f"Took {time() - t} s")
cuda_calc

Calculating the demagnetization tensor
(3, 1, 1, 6)
Took 1.0116775035858154 s


D:\Python\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
D:\Python\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (28) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[[[-6.36619760e-09, -0.00000000e+00, -0.00000000e+00,
          -6.36619760e-09, -0.00000000e+00, -6.36619761e-09]]],


       [[[ 2.30316545e+00, -6.36619770e-09, -6.36619773e-09,
          -1.15158275e+00,  2.85470179e-17, -1.15158275e+00]]],


       [[[ 2.30316545e+00,  6.36619773e-09,  6.36619773e-09,
          -1.15158275e+00,  2.85470179e-17, -1.15158275e+00]]]])

array([[[[-6.36619760e-09, -0.00000000e+00, -0.00000000e+00,
          -6.36619760e-09, -0.00000000e+00, -6.36619761e-09]]]])

In [85]:
diff = cuda_calc - tensor_original